In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
import csv 

prefixesUpper = ['HOU', 'LAA', 'OAK', 'SEA', 'TEX', 'BAL', 'BOS', 'NYY', 'TBR', 'TOR', 'CHW', 'CLE', 'DET', 'KCR', 'MIN', 'ATL', 'MIA', 'NYM', 'PHI', 'WSN', 'CHC', 'CIN', 'MIL', 'PIT', 'STL', 'ARI', 'COL', 'LAD', 'SDP', 'SFG']
prefixesLower = ['hou', 'laa', 'oak', 'sea', 'tex', 'bal', 'bos', 'nyy', 'tbr', 'tor', 'chw', 'cle', 'det', 'kcr', 'min', 'atl', 'mia', 'nym', 'phi', 'wsn', 'chc', 'cin', 'mil', 'pit', 'stl', 'ari', 'col', 'lad', 'sdp', 'sfg']

pathToCsv = "C:/Users/jeb/Repos/cypress-data-fetching/data/gamedata/ari-games.csv"

def makeGameCsvPaths(list):
    csvPaths = []
    for prefix in list:
        currentPath = f"C:/data/2022/game/{prefix}-games.csv"
        csvPaths.append(currentPath)
    return csvPaths

def makeBattingCsvPaths(list):
    csvPaths = []
    for prefix in list:
        currentPath = f"C:/data/2022/batting/{prefix}-batting-2022.csv"
        csvPaths.append(currentPath)
    return csvPaths

        
def readCsvPaths(paths):
    dataFromCsvList = []
    # can add columnsToDrop = ['Date', 'Time', 'Attendance'] and then in append data.drop(columnsToDrop, axis=1)
    for path in paths:
        currentCsvData = pd.read_csv(path)
        dataFromCsvList.append(currentCsvData)
    return pd.concat(dataFromCsvList)

gameDataFiles = makeGameCsvPaths(prefixesLower)
battingDataFiles = makeBattingCsvPaths(prefixesLower)
data = readCsvPaths(gameDataFiles)


#iterate_csv_data(gameData)


In [6]:
# Replace 'W' with 1 and 'L' with 0
print(data)
data = data.replace({'W': 1, 'W-wo':1, 'L': 0, 'L-wo': 0})
# Preprocess the data
X = data[['R', 'RA', 'Inn', 'Tm']] # replace with the names of the relevant features
y = data['W/L'] # replace with the name of the label column
X = (X - X.mean()) / X.std() # standardize the features
y = y.values.reshape(-1, 1) # reshape the label to a column vector
X = X.astype('float32') # convert X to float32
y = tf.convert_to_tensor(y, dtype=tf.float32)

# Split the data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

     Gm#             Date Unnamed: 2   Tm Unnamed: 4  Opp   W/L   R  RA   Inn  \
0      1   Thursday Apr 7   boxscore  HOU          @  LAA     W   3   1   NaN   
1      2     Friday Apr 8   boxscore  HOU          @  LAA     W  13   6   NaN   
2      3   Saturday Apr 9   boxscore  HOU          @  LAA     L   0   2   NaN   
3      4    Sunday Apr 10   boxscore  HOU          @  LAA     W   4   1   NaN   
4      5   Tuesday Apr 12   boxscore  HOU          @  ARI     W   2   1   NaN   
..   ...              ...        ...  ...        ...  ...   ...  ..  ..   ...   
157  158   Saturday Oct 1   boxscore  SFG        NaN  ARI     L   4   8   NaN   
158  159     Sunday Oct 2   boxscore  SFG        NaN  ARI  W-wo   4   3  10.0   
159  160     Monday Oct 3   boxscore  SFG          @  SDP     L   4   7   NaN   
160  161    Tuesday Oct 4   boxscore  SFG          @  SDP     L   2   6   NaN   
161  162  Wednesday Oct 5   boxscore  SFG          @  SDP     W   8   1   NaN   

     ...      GB          W

C:\Users\jeb\AppData\Local\Temp\ipykernel_13712\1436152909.py:7: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X = (X - X.mean()) / X.std() # standardize the features
C:\Users\jeb\AppData\Local\Temp\ipykernel_13712\1436152909.py:7: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X = (X - X.mean()) / X.std() # standardize the features


Epoch 1/10
122/122 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 2/10
122/122 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 3/10
122/122 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 4/10
122/122 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 5/10
122/122 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 6/10
122/122 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 7/10
122/122 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5077 - val_loss: nan - val_accuracy: 0.4691
Epoch 8/10
122/122 [============================